In [1]:
import pickle
import numpy as np

bsnipdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'
bsnipdemo = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))
bsniptopdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/BSNIP/'

fc = []
aps20 = []
aps15 = []
aps10 = []
aps5 = []
aps3 = []
aps1 = []
age = []
sex = []
race = []
sz = []

for sub in bsnipdemo['Age_cal']:
    try:
        a = bsnipdemo['Age_cal'][sub]
        s = bsnipdemo['sex'][sub]
        r = bsnipdemo['Race'][sub]
        d = bsnipdemo['DXGROUP_1'][sub]
        if d not in ['NC', 'SZP']:
            continue
        if r not in ['AA', 'CA']:
            continue
        s = s == 's1.0'
        r = r == 'AA'
        d = d == 'SZP'
        age.append(a)
        sex.append(s)
        race.append(r)
        sz.append(d)
        fc.append(np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy'))
        aps20.append(np.load(f'{bsniptopdir}/Top20/{sub}_task-unktop20_fc.npy'))
        aps3.append(np.load(f'{bsniptopdir}/Top3/{sub}_task-unktop3_fc.npy'))
        aps1.append(np.load(f'{bsniptopdir}/Top1/{sub}_task-unktop1_fc.npy'))
    except:
        pass

age = np.stack(age)
sex = np.stack(sex).astype('int')
race = np.stack(race).astype('int')
sz = np.stack(sz).astype('int')
fc = np.stack(fc)
aps20 = np.stack(aps20)
aps1 = np.stack(aps1)

print([x.shape for x in [age, sex, race, sz, fc, aps20, aps1]])

[(405,), (405,), (405,), (405,), (405, 34716), (405, 34716), (405, 34716)]


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression

class MLP(nn.Module):
    def __init__(self, out):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(34716,30).float().cuda()
        self.fc2 = nn.Linear(30,out).float().cuda()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze()
        return x

class GCN(nn.Module):
    def __init__(self, out):
        super(GCN, self).__init__()
        self.fc1 = nn.Linear(34716,30).float().cuda()
        self.fc2 = nn.Linear(30,out).float().cuda()

    def forward(self, x, E):
        x = F.relu(self.fc1(x))
        x = x.squeeze()
        x = E@x
        x = self.fc2(x)
        x = x.squeeze()
        return x

def rmse(yhat, y):
    return np.mean((yhat-y)**2)**0.5

def rmse2(yhat, y):
    return torch.mean((yhat-y)**2)**0.5

accs1 = []
accs2 = []
accs3 = []

for _ in range(20):

    xtr, xt, ytr, yt, vtr, vt, wtr, wt = train_test_split(fc, sz, aps20, aps1, stratify=sz, train_size=0.8)

    # rega = Ridge(alpha=1).fit(vtr, ytr)
    # regb = Ridge(alpha=1).fit(vtr-wtr, ytr)
    rega = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    regb = LogisticRegression(max_iter=1000).fit(xtr-wtr, ytr)

    # yhata = rega.predict(vt)
    # yhatb = regb.predict(vt-wt)
    yhata = rega.predict_proba(xt)
    yhatb = regb.predict_proba(xt-wt)

    yhat = (yhata+yhatb)/2
    # acc = rmse(yhat, yt)
    yhat = np.argmax(yhat, axis=1)
    acc = np.mean(yhat == yt)
    accs1.append(acc)
    print(acc)

    # mlp = MLP(2)
    # gcn = GCN(2)

    # opa = torch.optim.Adam(mlp.parameters(), lr=1e-4, weight_decay=1e-4)
    # opb = torch.optim.Adam(gcn.parameters(), lr=1e-4, weight_decay=1e-4)

    # Ntr = len(xtr)
    # Nt = len(xt)
    
    # Etr = torch.ones(Ntr, Ntr)/Ntr + torch.eye(Ntr)
    # Etr = Etr.float().cuda()
    
    # Et = torch.ones(Ntr+Nt, Ntr+Nt)/(Ntr+Nt) + torch.eye(Ntr+Nt)
    # Et = Et.float().cuda()

    # nepochs = 5000
    # pperiod = 500
    # ce = nn.CrossEntropyLoss()
    
    # xtr = torch.from_numpy(xtr).float().cuda()
    # ytr = torch.from_numpy(ytr).long().cuda()
    # xt = torch.from_numpy(xt).float().cuda()
    # yt = torch.from_numpy(yt).long().cuda()

    # for e in range(nepochs):
    #     opa.zero_grad()
    #     opb.zero_grad()
    #     yhata = mlp(xtr)
    #     yhatb = gcn(xtr, Etr)
    #     # lossa = rmse2(yhata, ytr)
    #     # lossb = rmse2(yhatb, ytr)
    #     lossa = ce(yhata, ytr)
    #     lossb = ce(yhatb, ytr)
    #     lossa.backward()
    #     lossb.backward()
    #     opa.step()
    #     opb.step()
    #     if e % pperiod == 0 or e == nepochs-1:
    #         print(f'{e} {float(lossa)} {float(lossb)}')

    # with torch.no_grad():
    #     yhata = mlp(xt)
    #     yhatb = gcn(torch.cat([xtr, xt], axis=0), Et)[Ntr:]
    #     # acca = float(rmse2(yhata, yt))
    #     # accb = float(rmse2(yhatb, yt))
    #     yhata = torch.argmax(yhata, dim=1).detach().cpu().numpy()
    #     yhatb = torch.argmax(yhatb, dim=1).detach().cpu().numpy()
    #     acca = np.mean(yhata == yt.detach().cpu().numpy())
    #     accb = np.mean(yhatb == yt.detach().cpu().numpy())
    #     print(acca)
    #     print(accb)
    #     accs2.append(acca)
    #     accs3.append(accb)

print('---')
print(np.mean(accs1), np.std(accs1))
# print(np.mean(accs2), np.std(accs2))
# print(np.mean(accs3), np.std(accs3))

0.7654320987654321
0.7777777777777778
0.8148148148148148
0.7777777777777778
0.7283950617283951
0.7283950617283951
0.7530864197530864
0.7530864197530864
0.7283950617283951
0.8395061728395061
0.8024691358024691
0.7160493827160493
0.8641975308641975
0.7901234567901234
0.8271604938271605
0.7654320987654321
0.6790123456790124
0.7407407407407407
0.7530864197530864
0.7777777777777778
---
0.7691358024691358 0.044013687580187366
